### Basic Setup
Here we check the sage version, configure some settings, define a 2-manifold $M$ with cylidrical coordinate chart $Y$, and a riemannian metric. The naming convention python variables (not variables used in the actual expressions) is `CamelCase` for objects defined on the manifold, and `snake_case` (with a trailing underscore for functions) for generic symbolic expressions. For rank $(0, 2)$ tensors, we default to the purely covariant form (as that is what we have regularization conditions for), denote the $(1, 1)$ version as `X_mat` and denote the rank $(2, 0)$ version as `X_con`.

In [8]:
# Check version
version()

'SageMath version 10.3, Release Date: 2024-03-19'

In [9]:
# Reset
reset()
# Setup pretty printing
%display latex

In [10]:
# Define a differentiable manifold of dimension 2 over real numbers
M = Manifold(2, 'M', latex_name=r'\mathcal{M}', start_index=1)
# Define a cylindrical chart on the manifold
Chart.<rho, z> = M.chart(r'rho:\rho z:z')
# Cache frame
Chartf = Chart.frame()

In [11]:
# Declare a metric on the manifold
G = M.riemannian_metric('g')

# Components of metric
s_ = function('seed', latex_name='S')(rho, z)
psi_ = function('psi', latex_name=r'\psi')(rho, z)

# Metric is symmetric
conformal_ = psi_**4 * exp(2 * rho * s_)

G[1, 1], G[2, 2] = conformal_, conformal_
G[1, 2] = 0

# Invert metric
G_con = G.inverse()

# Print metric
G.display()

g = e^(2*rho*seed(rho, z))*psi(rho, z)^4 drho⊗drho + e^(2*rho*seed(rho, z))*psi(rho, z)^4 dz⊗dz

In [12]:
G_con.display()

inv_g = e^(-2*rho*seed(rho, z))/psi(rho, z)^4 ∂/∂rho⊗∂/∂rho + e^(-2*rho*seed(rho, z))/psi(rho, z)^4 ∂/∂z⊗∂/∂z

In [13]:
# Compute Ricci Tensor
R = G.ricci(Chartf)
# And Ricci Scalar
R_trace = G_con['^ij'] * R['_ij']
# And connection coefficients, used for covariant derivative
Nabla = G.connection(name='nabla', latex_name=r'\nabla')
# Make sure metric is compatible
assert(Nabla(G) == 0)

Nabla.display()

Gam^rho_rho,rho = (rho*psi(rho, z)*d(seed)/drho + psi(rho, z)*seed(rho, z) + 2*d(psi)/drho)/psi(rho, z) 
Gam^rho_rho,z = (rho*psi(rho, z)*d(seed)/dz + 2*d(psi)/dz)/psi(rho, z) 
Gam^rho_z,rho = (rho*psi(rho, z)*d(seed)/dz + 2*d(psi)/dz)/psi(rho, z) 
Gam^rho_z,z = -(rho*psi(rho, z)*d(seed)/drho + psi(rho, z)*seed(rho, z) + 2*d(psi)/drho)/psi(rho, z) 
Gam^z_rho,rho = -(rho*psi(rho, z)*d(seed)/dz + 2*d(psi)/dz)/psi(rho, z) 
Gam^z_rho,z = (rho*psi(rho, z)*d(seed)/drho + psi(rho, z)*seed(rho, z) + 2*d(psi)/drho)/psi(rho, z) 
Gam^z_z,rho = (rho*psi(rho, z)*d(seed)/drho + psi(rho, z)*seed(rho, z) + 2*d(psi)/drho)/psi(rho, z) 
Gam^z_z,z = (rho*psi(rho, z)*d(seed)/dz + 2*d(psi)/dz)/psi(rho, z)

### Variables
Next we define the relavent tensor fields, as well as their components in our default frame.

In [14]:
# Resulting Tensors
Lam = M.scalar_field(rho * psi_^2, chart=Chart, name='lam', latex_name=r'\lambda')

LamLog = Nabla(Lam) / Lam
LamLog2 = Nabla(Nabla(Lam)) / Lam
           
Lam.display()

lam: M → ℝ
   (rho, z) ↦ rho*psi(rho, z)^2

In [15]:
# Gauge Fields
lapse_ = function('lapse', latex_name=r'\alpha')(rho, z)

shiftr_ = function('shiftr', latex_name=r'\beta^r')(rho, z)
shiftz_ = function('shiftz', latex_name=r'\beta^z')(rho, z)

# Gauge Tensors
Lapse = M.scalar_field(lapse_, chart=Chart, name='Lapse', latex_name=r'\alpha')

LapseLog = Nabla(Lapse) / Lapse
LapseLog2 = Nabla(Nabla(Lapse)) / Lapse

Shift = M.tensor_field(1, 0, name='Shift', latex_name=r'\beta')
Shift.add_comp(Chartf)[1] = shiftr_
Shift.add_comp(Chartf)[2] = shiftz_

Lapse.display()

Lapse: M → ℝ
   (rho, z) ↦ lapse(rho, z)

In [16]:
Shift.display()

Shift = shiftr(rho, z) ∂/∂rho + shiftz(rho, z) ∂/∂z

In [28]:
# Extrinsic Curvature

# Components
# krr_ = function('Krr_f', latex_name='K_{rr}')(rho, z)
# kzz_ = function('Kzz_f', latex_name='K_{zz}')(rho, z)
# krz_ = function('Krz_f', latex_name='K_{rz}')(rho, z)

krr_ = SR(0)
kzz_ = SR(0)
krz_ = SR(0)

# Phi Component of K
# y_ = function('y_f', latex_name=r'Y')(rho, z)
y_ = SR(0)


K = M.tensor_field(0, 2, sym=(0, 1), name='K')
K.add_comp(Chartf)[1, 1] = krr_
K.add_comp(Chartf)[2, 2] = kzz_
K.add_comp(Chartf)[1, 2] = krz_

K_mat = G_con['^{ij}'] * K['_jk']
K_con = K_mat['^i_k'] * G_con['^{kj}']
K_trace = G_con['^ij'] * K['_ij']

# L = K_\phi^\phi
L = M.scalar_field(krr_/conformal_ + rho * y_, name='L')

K.display()

K = 0

In [29]:
L.display()

L: M → ℝ
   (rho, z) ↦ 0

### Constraint equations
Constraint equations that should be satisfied at all times.

In [33]:
term1 = (K_trace^2 - K['_ij'] * K_con['^ij'] + R_trace) / 2
term2 = - LamLog2['_ij'] * G_con['^{ij}'] + K_trace * L

# Hamiltonian
CH = term1 + term2

(-CH.expr() * conformal_ * psi_ / 4).expand()

1/4*rho*psi(rho, z)*diff(seed(rho, z), rho, rho) + 1/4*rho*psi(rho, z)*diff(seed(rho, z), z, z) + 1/2*psi(rho, z)*diff(seed(rho, z), rho) + diff(psi(rho, z), rho)/rho + diff(psi(rho, z), rho, rho) + diff(psi(rho, z), z, z)

In [35]:
term1 = Nabla(K_mat)['^i_ji'] - Nabla(K_trace + L)
term2 = LamLog['_j'] * K_mat['^j_i'] - LamLog * L

# Momentum Constraint
CM = term1 + term2

CM.display()

0

### Evolution Equations
Calculations for $\mathcal{L}_n X$ for various dynamical quantities.

In [41]:
term1 = R - LamLog2 - LapseLog2
term2 = (K_trace + L) * K - 2 * K['_ij'] * K_mat['^j_k']

# Extrinsic Curvature
LieK = term1 + term2

LieK.display()

-(rho^2*lapse(rho, z)*psi(rho, z)^2*d^2(seed)/drho^2 + rho^2*lapse(rho, z)*psi(rho, z)^2*d^2(seed)/dz^2 - rho*psi(rho, z)^2*seed(rho, z)*d(lapse)/drho - lapse(rho, z)*psi(rho, z)^2*seed(rho, z) + rho*psi(rho, z)^2*d^2(lapse)/drho^2 - 4*rho*lapse(rho, z)*(d(psi)/drho)^2 + 4*rho*lapse(rho, z)*psi(rho, z)*d^2(psi)/drho^2 + 2*rho*psi(rho, z)*d(lapse)/dz*d(psi)/dz + 2*rho*lapse(rho, z)*d(psi)/dz^2 + 2*rho*lapse(rho, z)*psi(rho, z)*d^2(psi)/dz^2 - 2*(rho*lapse(rho, z)*psi(rho, z)*seed(rho, z) + rho*psi(rho, z)*d(lapse)/drho - lapse(rho, z)*psi(rho, z))*d(psi)/drho - (rho^2*psi(rho, z)^2*d(lapse)/drho + 2*rho^2*lapse(rho, z)*psi(rho, z)*d(psi)/drho - rho*lapse(rho, z)*psi(rho, z)^2)*d(seed)/drho + (rho^2*psi(rho, z)^2*d(lapse)/dz + 2*rho^2*lapse(rho, z)*psi(rho, z)*d(psi)/dz)*d(seed)/dz)/(rho*lapse(rho, z)*psi(rho, z)^2) drho⊗drho + (psi(rho, z)^2*seed(rho, z)*d(lapse)/dz - psi(rho, z)^2*d^2(lapse)/drhodz + 2*psi(rho, z)*d(lapse)/dz*d(psi)/drho - 2*lapse(rho, z)*psi(rho, z)*d^2(psi)/drhodz + 2*(lapse(rho, z)*psi(rho, z)*seed(rho, z) + psi(rho, z)*d(lapse)/drho + 3*lapse(rho, z)*d(psi)/drho)*d(psi)/dz + (rho*psi(rho, z)^2*d(lapse)/dz + 2*rho*lapse(rho, z)*psi(rho, z)*d(psi)/dz)*d(seed)/drho + (rho*psi(rho, z)^2*d(lapse)/drho + 2*rho*lapse(rho, z)*psi(rho, z)*d(psi)/drho + lapse(rho, z)*psi(rho, z)^2)*d(seed)/dz)/(lapse(rho, z)*psi(rho, z)^2) drho⊗dz + (psi(rho, z)^2*seed(rho, z)*d(lapse)/dz - psi(rho, z)^2*d^2(lapse)/drhodz + 2*psi(rho, z)*d(lapse)/dz*d(psi)/drho - 2*lapse(rho, z)*psi(rho, z)*d^2(psi)/drhodz + 2*(lapse(rho, z)*psi(rho, z)*seed(rho, z) + psi(rho, z)*d(lapse)/drho + 3*lapse(rho, z)*d(psi)/drho)*d(psi)/dz + (rho*psi(rho, z)^2*d(lapse)/dz + 2*rho*lapse(rho, z)*psi(rho, z)*d(psi)/dz)*d(seed)/drho + (rho*psi(rho, z)^2*d(lapse)/drho + 2*rho*lapse(rho, z)*psi(rho, z)*d(psi)/drho + lapse(rho, z)*psi(rho, z)^2)*d(seed)/dz)/(lapse(rho, z)*psi(rho, z)^2) dz⊗drho - (rho^2*lapse(rho, z)*psi(rho, z)^2*d^2(seed)/drho^2 + rho^2*lapse(rho, z)*psi(rho, z)^2*d^2(seed)/dz^2 + rho*psi(rho, z)^2*seed(rho, z)*d(lapse)/drho + lapse(rho, z)*psi(rho, z)^2*seed(rho, z) + rho*psi(rho, z)^2*d^2(lapse)/dz^2 + 2*rho*lapse(rho, z)*(d(psi)/drho)^2 + 2*rho*lapse(rho, z)*psi(rho, z)*d^2(psi)/drho^2 - 2*rho*psi(rho, z)*d(lapse)/dz*d(psi)/dz - 4*rho*lapse(rho, z)*d(psi)/dz^2 + 4*rho*lapse(rho, z)*psi(rho, z)*d^2(psi)/dz^2 + 2*(rho*lapse(rho, z)*psi(rho, z)*seed(rho, z) + rho*psi(rho, z)*d(lapse)/drho + lapse(rho, z)*psi(rho, z))*d(psi)/drho + (rho^2*psi(rho, z)^2*d(lapse)/drho + 2*rho^2*lapse(rho, z)*psi(rho, z)*d(psi)/drho + 3*rho*lapse(rho, z)*psi(rho, z)^2)*d(seed)/drho - (rho^2*psi(rho, z)^2*d(lapse)/dz + 2*rho^2*lapse(rho, z)*psi(rho, z)*d(psi)/dz)*d(seed)/dz)/(rho*lapse(rho, z)*psi(rho, z)^2) dz⊗dz

In [39]:
# Metric and Lambda
LieG = -2*K
LieLam = -Lam * L

LieLam.display()

dzero(∂/∂z): M → ℝ
   (rho, z) ↦ 0